In [105]:
# %pip install git+https://github.com/tensorflow/docs

import requests
import os.path
import asyncio
import json
import numpy as np
import pathlib
import pandas as pd
import seaborn as sns
import tensorflow as tf
import time
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
from riotwatcher import TftWatcher, ApiError

from utils.configuration import settings
from utils.utils import *
from tft.api import *


In [106]:
API_KEY = settings.api_key
ASSETS_DIR = settings.assets_dir
SERVER = 'na1'  # euw1 na1 kr oc1
LEAGUE='challengers'
MAX_COUNT = settings.max_count
LATEST_RELEASE = '12.12.450.4196'

In [107]:
summoners_df = pd.read_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_summoners.pickle'))

In [108]:
summoners_df

,id,accountId,puuid,name,profileIconId,revisionDate,summonerLevel,summonerId,summonerName,leaguePoints,rank,wins,losses,veteran,inactive,freshBlood,hotStreak
0,2xW2jSNMMQPOcF3lg8m7TtjXc38a5-KtWxGDiOvrgOLSQfo,F9GYe1tYcrU9CjqlyJRJVzobPX1zXjOF6rzBjBMtyx8sfLU,jgoRVbpnne2Mr7wg1lpDJ0_o9uVu6v9ryvMzEgTjlDy9VW...,ChungMoney12,4025,1656308821000,208,2xW2jSNMMQPOcF3lg8m7TtjXc38a5-KtWxGDiOvrgOLSQfo,ChungMoney12,611,I,134,78,False,False,False,False
1,9BtCw3yqmbey6yGARkIAPCeKeCPF4SqzrwlAtbuFVTpX06s,GxpOZtrCOBmQGwVK-lpxsiE5-YaVPi3Mb2WoaqpiKsiGuA,5EBycu8MnY8t5LmyomNOoVcFt54BEvMeiguhbjrHxFy7_g...,Sphinx,576,1656373488000,72,9BtCw3yqmbey6yGARkIAPCeKeCPF4SqzrwlAtbuFVTpX06s,Sphinx,838,I,149,88,False,False,True,False
2,O5jsqhLy0mX6raPFx0wo1wnZCTUs86-FL3tDd3_9Nq1TTdUE,ersZ7nn2iDcLYBFW5YrvKW1WXJVne_nwk2u99957Xe-V84...,FEFuaQWa6yJM8q-VhDOh6TopPwhxNg8OCXBc9xi_c0R2Yt...,NG Tae,3542,1656362983000,25,O5jsqhLy0mX6raPFx0wo1wnZCTUs86-FL3tDd3_9Nq1TTdUE,NG Tae,563,I,175,143,False,False,True,False
3,1jPWoCYQSUgKZeHXT9UIfyXqEYNaDN5vx6LLd8blCDaztpoV,RIzLPt0Yj1Nmx_V91Nq41KkK0jw0N8H6IJTCs3Smthlmc3...,XWwLJnEh-o7p_WMIa2mmoyRKwJoX9bDMucuEsb2PqrSZUK...,Nostereleven,4834,1656404629000,191,1jPWoCYQSUgKZeHXT9UIfyXqEYNaDN5vx6LLd8blCDaztpoV,Nostereleven,753,I,148,108,False,False,True,False
4,MFA9xDr1qpMHq3p1P3Kd7yQRjmVZZeKgiEGMNOZaoHvSS_A,vNJAtsWbXGLyDPv5HWK6T2VsVvgaY3O7LOUNvL23vJt1-Q,hkhRHja9YNEGxs-c2YZxHJmvTcZaPQSaAMHWnbYt0qwlEC...,Pawnup,659,1656378493000,140,MFA9xDr1qpMHq3p1P3Kd7yQRjmVZZeKgiEGMNOZaoHvSS_A,Pawnup,731,I,164,111,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80,m-qUxgZJLteV8W5Oqwwn50XfK0wXDtalCaVm9reptusp2JY,HG1ubDWkjeBciVDvB0bzOVDPf1SNPNt6Pk23ctu6fyQihA,WhTl_YwHuw06oZYoBvabD1I06ZBEUcelhXUlBDNnPXSKn-...,milala,3838,1656400235000,403,m-qUxgZJLteV8W5Oqwwn50XfK0wXDtalCaVm9reptusp2JY,milala,566,I,159,128,False,False,True,False
81,hpVP56ElzFpI9hJtd7l53WcBMs8s2kvqDEiWJnGCJVyLnSg,4Yvk3S970wp_QRtVv1yJ-TgydOOk_qZHnO_LscPrurKPhA,XqIjpzF_Iq4c05mQ3t8f5R7RUyrWxJVdqL3AoyiyUxg9uI...,FingerLicker,4293,1656391166000,282,hpVP56ElzFpI9hJtd7l53WcBMs8s2kvqDEiWJnGCJVyLnSg,FingerLicker,697,I,174,117,False,False,True,False
82,NI1BS8WAKRRk2qPubjPi3XGdv9xaKjPzJQuAzpie33cm-Fxq,qgL9DKswEAkNWKYHk6B4AO2-tlw4a3PfJdf0t0FLPjKT5n...,P6tuguVlHjl_9KVogt65c9N9Q1Hq34UMLAE2pqnweLIQKM...,OatmealWater,5271,1656412837000,48,NI1BS8WAKRRk2qPubjPi3XGdv9xaKjPzJQuAzpie33cm-Fxq,OatmealWater,595,I,163,114,False,False,True,False
83,BagRHiTwUX1JDU4tWpzTHx_QTlT-GIUea6YSFiji3Q_crjs,D_5UbgHe9hyxxZYbTlNDdelEQ4Y2-zzbQDWdS3YLO1dbNFM,lCUpY-VbMzt1RZ42VUmI0S3799eXOoHRQArRXXS5UeRg0j...,daCableguy,983,1656304188000,143,BagRHiTwUX1JDU4tWpzTHx_QTlT-GIUea6YSFiji3Q_crjs,daCableguy,506,I,134,105,False,False,True,True


In [109]:

# Get all unique matches_id from assets dir
matches_asset = load_matches(summoners_df, server=SERVER)
matches_id = [match['metadata']['match_id'] for match in matches_asset]
seen = set()
uniq_matches_id = [
    x for x in matches_id if x not in seen and not seen.add(x)]

In [110]:
len(uniq_matches_id)

2870

In [111]:
len(matches_asset)

3408

In [112]:
seen = set()
seen_add = seen.add
uniq_matches = [x for x in matches_asset if x['metadata']['match_id'] not in seen and not seen_add(x['metadata']['match_id'])]

In [113]:
len(uniq_matches)

2870

In [114]:
matches_asset[0]['info']['game_version']

'Version 12.11.446.9344 (Jun 09 2022/13:23:39) [PUBLIC] <Releases/12.11>'

In [115]:
latest_matches = [match for match in uniq_matches if LATEST_RELEASE in match['info']['game_version']]

In [116]:
len(latest_matches)

951

In [117]:
# latest_matches[0]['info']

In [118]:
matches_array = []

for match_row in latest_matches:
    match_id = match_row['metadata']['match_id']

    for participant in match_row['info']['participants']:
        match = {}
        match['match_id'] = match_id
        # match['level'] = participant['level']
        match['placement'] = participant['placement']
        # match['players_eliminated'] = participant['players_eliminated']
        # match['total_damage_to_players'] = participant['total_damage_to_players']

        for augment_index, augment in enumerate(participant['augments']):
            match[f'augment{augment_index}'] = augment

        for trait_index, trait in enumerate(participant['traits']):
            match[f'{trait["name"]}'] = trait["tier_current"]

        for unit_index, unit in enumerate(participant['units']):
            match[f'{unit["character_id"]}'] = unit["tier"]
            for item_index, item in enumerate(unit['itemNames']):
                match[f'{unit["character_id"]}_item{item_index}'] = item

        matches_array.append(match)

In [119]:
matches_array[0]
# len(matches_array)

{'match_id': 'NA1_4353652791',
 'placement': 1,
 'augment0': 'TFT6_Augment_Weakspot',
 'augment1': 'TFT6_Augment_Electrocharge2',
 'augment2': 'TFT6_Augment_ComponentGrabBag',
 'Set7_Bard': 1,
 'Set7_Bruiser': 1,
 'Set7_Cavalier': 0,
 'Set7_Dragon': 1,
 'Set7_Dragonmancer': 0,
 'Set7_Guild': 2,
 'Set7_Jade': 0,
 'Set7_Legend': 0,
 'Set7_Mirage': 0,
 'Set7_Mystic': 0,
 'Set7_Ragewing': 2,
 'Set7_Shapeshifter': 1,
 'Set7_Swiftshot': 1,
 'Set7_Tempest': 0,
 'Set7_Warrior': 1,
 'TFT7_Shen': 2,
 'TFT7_Twitch': 2,
 'TFT7_Twitch_item0': 'TFT_Item_MadredsBloodrazor',
 'TFT7_Twitch_item1': 'TFT_Item_InfinityEdge',
 'TFT7_Twitch_item2': 'TFT_Item_GuinsoosRageblade',
 'TFT7_Hecarim': 2,
 'TFT7_Hecarim_item0': 'TFT_Item_ThiefsGloves',
 'TFT7_Hecarim_item1': 'TFT_Item_SeraphsEmbrace',
 'TFT7_Hecarim_item2': 'TFT_Item_HextechGunblade',
 'TFT7_Neeko': 2,
 'TFT7_Neeko_item0': 'TFT_Item_GargoyleStoneplate',
 'TFT7_Xayah': 2,
 'TFT7_Xayah_item0': 'TFT_Item_Quicksilver',
 'TFT7_Xayah_item1': 'TFT_Item_Gu

In [120]:
matches_league_df = pd.json_normalize(matches_array)

In [121]:
matches_league_df

,match_id,placement,augment0,augment1,augment2,Set7_Bard,Set7_Bruiser,Set7_Cavalier,Set7_Dragon,Set7_Dragonmancer,...,TFT7_Aatrox_item1,TFT7_Vladimir_item1,TFT7_Vladimir_item2,TFT7_Thresh_item2,TFT7_Senna_item0,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Skarner_item1,TFT7_Skarner_item2,TFT7_Aatrox_item2
0,NA1_4353652791,1,TFT6_Augment_Weakspot,TFT6_Augment_Electrocharge2,TFT6_Augment_ComponentGrabBag,1.0,1.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4353652791,8,TFT6_Augment_CelestialBlessing1,TFT6_Augment_SecondWind2,TFT7_Augment_RevelPartyFavors,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4353652791,4,TFT7_Augment_BruiserTrait,TFT6_Augment_SecondWind2,TFT6_Augment_TomeOfTraits1,NaN,2.0,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4353652791,3,TFT7_Augment_RagewingTrait,TFT6_Augment_TargetDummies,TFT6_Augment_CelestialBlessing2,NaN,0.0,0.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4353652791,7,TFT6_Augment_TriForce1,TFT6_Augment_Meditation2,TFT7_Augment_TempestEyeOfTheStorm,NaN,0.0,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7603,NA1_4355169385,8,TFT6_Augment_Weakspot,TFT7_Augment_SwiftshotPressTheAttack,TFT7_Augment_UrfsGrabBag1,NaN,1.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7604,NA1_4355169385,6,TFT7_Augment_GuildTrait,TFT6_Augment_Ascension,TFT7_Augment_CavalierEmblem,NaN,1.0,1.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7605,NA1_4355169385,5,TFT6_Augment_ItemGrabBag1,TFT6_Augment_SunfireBoard,TFT6_Augment_Ascension,1.0,1.0,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7606,NA1_4355169385,4,TFT6_Augment_Weakspot,TFT6_Augment_Meditation2,TFT6_Augment_SalvageBin,NaN,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [122]:
matches_league_df[matches_league_df.isnull().T.any()]

,match_id,placement,augment0,augment1,augment2,Set7_Bard,Set7_Bruiser,Set7_Cavalier,Set7_Dragon,Set7_Dragonmancer,...,TFT7_Aatrox_item1,TFT7_Vladimir_item1,TFT7_Vladimir_item2,TFT7_Thresh_item2,TFT7_Senna_item0,TFT7_Senna_item1,TFT7_Senna_item2,TFT7_Skarner_item1,TFT7_Skarner_item2,TFT7_Aatrox_item2
0,NA1_4353652791,1,TFT6_Augment_Weakspot,TFT6_Augment_Electrocharge2,TFT6_Augment_ComponentGrabBag,1.0,1.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NA1_4353652791,8,TFT6_Augment_CelestialBlessing1,TFT6_Augment_SecondWind2,TFT7_Augment_RevelPartyFavors,1.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NA1_4353652791,4,TFT7_Augment_BruiserTrait,TFT6_Augment_SecondWind2,TFT6_Augment_TomeOfTraits1,NaN,2.0,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NA1_4353652791,3,TFT7_Augment_RagewingTrait,TFT6_Augment_TargetDummies,TFT6_Augment_CelestialBlessing2,NaN,0.0,0.0,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NA1_4353652791,7,TFT6_Augment_TriForce1,TFT6_Augment_Meditation2,TFT7_Augment_TempestEyeOfTheStorm,NaN,0.0,NaN,NaN,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7603,NA1_4355169385,8,TFT6_Augment_Weakspot,TFT7_Augment_SwiftshotPressTheAttack,TFT7_Augment_UrfsGrabBag1,NaN,1.0,0.0,1.0,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7604,NA1_4355169385,6,TFT7_Augment_GuildTrait,TFT6_Augment_Ascension,TFT7_Augment_CavalierEmblem,NaN,1.0,1.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7605,NA1_4355169385,5,TFT6_Augment_ItemGrabBag1,TFT6_Augment_SunfireBoard,TFT6_Augment_Ascension,1.0,1.0,0.0,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7606,NA1_4355169385,4,TFT6_Augment_Weakspot,TFT6_Augment_Meditation2,TFT6_Augment_SalvageBin,NaN,1.0,1.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [123]:
matches_league_df.to_pickle(os.path.join(ASSETS_DIR, f'{SERVER}_{LEAGUE}_matches.pickle'))